# Scripted Primo Search Assessment Tool

Description: < Describe the purpose of the SPSAT>

## Load required libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import glob
import os
import codecs
import json
import time
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus
import pymarc
import marcx
import io

## Define Some Variables

In [2]:
## URLs ##

# prod_url = <base url>
prod_url_base = 'http://bu-primo.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'
# stage_url = <base url>
stage_url_base = 'http://bu-primostage.hosted.exlibrisgroup.com:1701/PrimoWebServices/xservice/search/brief'

## query params ##
query_Params1 = '?institution=BOSU&query=any,contains,'
query_Params2 = '&indx=1&bulkSize=2'
query_Params3 = '&loc=local,scope:(BOSU)&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'
#query_Params3 = '&loc=local,scope:(ALMA_BOSU1)&loc=local,scope:(BOSU)&loc=adaptor,primo_central_multiple_fe&onCampus=true&json=true'

##&loc=local,scope:(ALMA_BOSU1)

stage = True
#production = True
#num_results = 10
#scope = 

## file locations ## 

# move to working directory, this is the path where we want files to reside
os.chdir('/Volumes/jwa_drive1/git/spst')



## Get Search Strings

In [3]:
### if loading search strings from a csv file
#### open input file, read each line as a separate string, and remove the new line character
# Input file (contains search strings)
f = '/Volumes/jwa_drive1/git/spst/search_strings.csv'
search_strings = open(f).read().splitlines()

### if building search strings from a marcxml file
# Input file (contains search strings)
#f = '/Volumes/jwa_drive1/git/spst/asl-1990.xml'
f = '/Volumes/jwa_drive1/git/spst/cr_records.xml'


records = pymarc.parse_xml_to_array(io.open(f,mode='r',encoding='utf-8'))
df_search = pd.DataFrame()

for rec in records:
    d = {}
    rec = marcx.FatRecord.from_record(rec)
    try:
        d['author'] = rec['100']['a']
    except Exception as e:
        d['author'] = ''
    d['title'] = rec.title() #.replace('/','')
    d['mmsid'] = rec['001'].data
    d['title_author'] = rec.title() #.replace('/','') + ' ' + d['author'] #rec.author()
    df_search = df_search.append(d,ignore_index=True)
title_search_strings = df_search['title'] 
title_author_search_strings = df_search['title_author'] 

In [4]:
#select the list of search strings
search_strings = title_search_strings
search_strings = title_author_search_strings
#search_strings = search_strings
#search_strings.head()

## Define functions

In [6]:
# build_url 
    # takes base_url and search_string, returns url properly formated for searching
def build_url(base_url,search_string):
    url = base_url+query_Params1 + search_string.replace(' ','+') + query_Params2 + query_Params3
    #print(url)
    return(url)
    
# parse_response 
    # takes response (as json) from search, returns response_dict (dict of our responses)
def parse_response(search,response,platform,search_no):
    response_dict = {}
    # define the elements to grab from the json response. These become the columns in the dataframe
    elements = []
    #elements.append('Platform')
    #elements.append('Search')
    #elements.append('ResultNumber')
    #elements.append('TotalHits')
    #elements.append('Rank')
    #elements.append('SearchEngine')
    elements.append('control.recordid')
    elements.append('display.type')
    elements.append('display.creator')
    elements.append('display.title')
    elements.append('display.ispartof')
    elements.append('delivery.delcategory')
    elements.append('delivery.fulltext')
    elements.append('search.searchscope')
    elements.append('search.general')
    elements.append('facets.collection')
    elements.append('facets.frbrtype')
    elements.append('facets.toplevel')
    elements.append('facets.prefilter')
    elements.append('sort')
    # parse the response to return the data desired elements
    response = json.loads(response.decode('utf8'))
    # need to extract the desired elements and add them to a dict
    response = response['SEGMENTS']
    response = response['JAGROOT']
    response = response['RESULT']
    response_dict['facet'] = response['FACETLIST']
    response_dict['docset'] = response['DOCSET']
    docs = response_dict['docset']['DOC']
    facets = response_dict['facet']
        
    ## define a dataframe to return the results
    doc_columns = ['Platform','Search_No','Search','ResultNumber','TotalHits','FirstHit','LastHit','Rank','SearchEngine','recordid', \
           'type','creator','title','ispartof','delcategory','fulltext','searchscope','general','collection', \
          'frbrtype','toplevel','prefilter','sort']
    df_doc = pd.DataFrame(columns = doc_columns)
    fac_columns = ['Platform','Search_No','Search','TotalHits','SearchEngine','facet','values']
    df_facets = pd.DataFrame(columns = fac_columns)
    #bib = doc['PrimoNMBib']['record']
    #print(type(bib)
    ## iterate through the results (normally 10) to populate a dictionary that will be added to the dataframe
    for doc in docs:
        doc_dict = {}
        doc_dict['Platform'] = platform
        doc_dict['Search_No'] = search_no
        doc_dict['Search'] = search
        doc_dict['ResultNumber'] = doc['@NO']
        doc_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
        doc_dict['FirstHit'] = response['DOCSET']['@FIRSTHIT']
        doc_dict['LastHit'] = response['DOCSET']['@LASTHIT']
        doc_dict['Rank'] = doc['@RANK']
        doc_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
        #doc_dict['Bib'] = doc['PrimoNMBib']['record']
        Bib = doc['PrimoNMBib']['record']
        
        for element in elements:
            x = element.split('.')
            if len(x) == 1:
                try:
                    doc_dict[x[-1]] = Bib[x[0]]
                except:
                    pass
            if len(x) == 2:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]]
                except:
                    pass
            if len(x) == 3:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]]
                except:
                    pass
            if len(x) == 4:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]][x[3]]
                except:
                    pass
            if len(x) == 5:
                try:
                    doc_dict[x[-1]] = Bib[x[0]][x[1]][x[2]][x[3]][x[4]]
                except:
                    pass   
        df_doc = df_doc.append(doc_dict, ignore_index=True)
        #print(doc_dict)
        f_dict = parse_facets(facets['FACET'])
        for k,v in f_dict.items() :
            if doc['@NO'] == '1':
                facet_dict = {}
                facet_dict['Platform'] = platform
                facet_dict['Search_No'] = search_no
                facet_dict['Search'] = search
                #facet_dict['ResultNumber'] = doc['@NO']
                facet_dict['TotalHits'] = response['DOCSET']['@TOTALHITS']
                facet_dict['SearchEngine'] = doc['@SEARCH_ENGINE']
                facet_dict['facet'] = k
                facet_dict['values'] = v
                df_facets = df_facets.append(facet_dict,ignore_index=True)

    return(df_doc,Bib,df_facets)


def parse_facets(facets):
    return_dict = {}
    for facet in facets:
        f = facet['@NAME']
        fv = facet['FACET_VALUES']
        #print(f)
        l = []
        for x in fv :
            try:
                l.append((x['@KEY'],int(x['@VALUE'])))
            except Exception as e:
                pass
        sorted_by_second = sorted(l, key=lambda tup: tup[1],reverse=True)
        return_dict[f] = sorted_by_second
    return return_dict

In [7]:

# create a DataFrame. Expand the columns to match the desired elements
# to be returned by the function 'parse_response'

# Create a dataframe with
# - column for search string 
# - column for response (returned from parse_response fuction)
columns = ['Platform','Search','ResultNumber','TotalHits','FirstHit','LastHit','Rank','SearchEngine','recordid', \
           'type','creator','title','ispartof','delcategory','fulltext','searchscope','general','collection', \
          'frbrtype','toplevel','prefilter','sort']
df = pd.DataFrame(columns = columns)
fac_columns = ['Platform','Search_No','Search','TotalHits','SearchEngine','facet','values']
df_f = pd.DataFrame(columns = fac_columns)

# Loop through search_strings, execute search, ... [to be done?]
counter = 0
for search_string in search_strings[:]:
    time.sleep(1)
    print(counter)
    #print(search_string)
    prod_search_url = build_url(prod_url_base,search_string)
    prod_request = Request(prod_search_url)
    stage_search_url = build_url(stage_url_base,search_string)
    stage_request = Request(stage_search_url)    
    try:
        response_body = urlopen(prod_request).read() # reads request into response_body var
        prod = response_body
        d = parse_response(search_string,response_body,'production',counter)[0]
        b = parse_response(search_string,response_body,'production',counter)[1]
        f = parse_response(search_string,response_body,'production',counter)[2]
        df = df.append(d, ignore_index=True)
        df_f = df_f.append(f,ignore_index=True)
        response_body = urlopen(stage_request).read() # reads request into response_body var
        stage = response_body
        d = parse_response(search_string,response_body,'stage',counter)[0]
        b = parse_response(search_string,response_body,'stage',counter)[1]
        f = parse_response(search_string,response_body,'stage',counter)[2]
        df = df.append(d, ignore_index=True)
        df_f = df_f.append(f,ignore_index=True)

    #prints errors 
    except Exception as e:
        pass
        #print('Exception: ',e)
    counter += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [10]:
search1 = df
search1 = search1.ix[:,'Platform':]
search1 = search1.sort_values(by=['Search_No','ResultNumber'], ascending=[1,1])

In [11]:
search1.shape

(9820, 21)

In [12]:
## here we are going to combine values from the search on production and stage to compare the results for each search
columns = ['Search','match','pRecordid','sRecordid','pRank','sRank','pCollection','sCollection', 
                        'pCreator','sCreator', 'pDelcategory','sDelcategory','pFrbrtype','sFrbrtype', 
                        'pFulltext','sFulltext', 'pTitle','sTitle', 'pToplevel','sToplevel', 'pType','sType']
comp_results = pd.DataFrame(columns = columns)
mask = search1['Platform'] == 'production'
production = search1[mask] 
mask = search1['Platform'] == 'stage'
stage = search1[mask]
stage= stage[['Rank', 'ResultNumber', 'Search', 'SearchEngine',
       'Search_No', 'TotalHits', 'collection', 'creator', 'delcategory',
       'frbrtype', 'fulltext', 'general', 'ispartof', 'prefilter', 'recordid',
       'searchscope', 'sort', 'title', 'toplevel', 'type']]
production = production[['Rank', 'ResultNumber', 'Search', 'SearchEngine',
       'Search_No', 'TotalHits', 'collection', 'creator', 'delcategory',
       'frbrtype', 'fulltext', 'general', 'ispartof', 'prefilter', 'recordid',
       'searchscope', 'sort', 'title', 'toplevel', 'type']]
stage= stage.set_index(['Search_No', 'ResultNumber'])
#cr_df = production[['pRecordid', 'ResultNumber','Search', 'pTitle', 'pCollection',  'pType','pFulltext','pDelcategory']]
cr_df = production.set_index(['Search', 'ResultNumber'])
production = production.set_index(['Search_No', 'ResultNumber'])
production = production[['recordid','Rank', 'Search','collection', 'creator','delcategory', 'frbrtype', 'fulltext','title', 'toplevel','type']]
stage = stage[['recordid','Rank', 'Search','collection', 'creator','delcategory', 'frbrtype', 'fulltext','title', 'toplevel','type']]
production.columns = ['pRecordid','pRank', 'Search', 'pCollection', 'pCreator', 'pDelcategory', 'pFrbrtype',
       'pFulltext', 'pTitle', 'pToplevel', 'pType']
stage.columns = ['sRecordid','sRank', 'Search', 'sCollection', 'sCreator', 'sDelcategory', 'sFrbrtype',
       'sFulltext', 'sTitle', 'sToplevel', 'sType']

In [13]:
if (stage.shape[1] == production.shape[1]):
    rows = set(range(0,min(stage.shape[0],production.shape[0])))
    cols = set(range(0,stage.shape[1]))
    scolumns = stage.columns
    pcolumns = production.columns
    for row in rows:
        d = {}
        for col in cols:
            if scolumns[col] == 'sRecordid':
                if stage.iloc[row][scolumns[col]] == production.iloc[row][pcolumns[col]]:
                    d['match'] = True
                else:
                    d['match'] = False
            d[scolumns[col]] = stage.iloc[row][scolumns[col]]
            d[pcolumns[col]] = production.iloc[row][pcolumns[col]]
        comp_results = comp_results.append(d,ignore_index=True)


In [14]:
# Define the aggregation calculations
aggregations = {
    'match' : {
        'match' : 'count'
    },
    'sType' : {
        'stype' : 'count'
    },
    'pType' : {
        'pType' : 'count'
        
    },

    'sToplevel' : {
        'pToplevel' : 'count'
        
    },
    'pToplevel' : {
        'pToplevel': 'count'
        
    }
}

In [15]:
by_search = comp_results.groupby(['Search']) #.agg(aggregations)


In [16]:
cr_df = cr_df[['recordid',  'title', 'collection', 'creator', 'type','fulltext','delcategory']]
cr_df

recordid  \
Search                                             ResultNumber                                                      
EXAMINING THE RELATIONSHIP BETWEEN SEXUAL OFFEN... 1                                   ALMA_BOSU151692162940001161   
                                                   2                          TN_tayfranc10.1080/13552600802056640   
CULTURE OF AFFLUENCE: PSYCHOLOGICAL COSTS OF MA... 1                                   ALMA_BOSU151692182360001161   
                                                   2                         TN_wj10.1046/j.1467-8624.2003.00625.x   
DATING VIOLENCE AMONG ADOLESCENTS.                 1                                   ALMA_BOSU151692179160001161   
                                                   2                                   ALMA_BOSU151692172810001161   
EXCEPTIONAL FEMALE STUDENTS OF COLOR.              1                                   ALMA_BOSU151692189290001161   
                                                   2                                               TN_ericEJ809148   
RELATIONSHIP QUALITY AND DOMESTIC VIOLENCE IN W... 1                         TN_wj10.1111/j.1471-6402.2005.00220.x   
                                                   2                                   ALMA_BOSU151692183260001161   
LANGUAGE BROKERING: AN INTEGRATIVE REVIEW OF TH... 1                                   ALMA_BOSU151692179430001161   
                                                   2                                               TN_ericEJ723424   
CHANGES IN PERSONAL RELIGION/SPIRITUALITY DURIN... 1                               TN_apa_articles10.1037/a0016211   
                                                   2                                   ALMA_BOSU151692179740001161   
COGNITIVE MECHANISMS UNDERLYING RECOVERED-MEMOR... 1                                   ALMA_BOSU151692175860001161   
                                                   2                                          TN_proquest195586098   
HOW CAN THEY LOOK SO HAPPY?                        1                                   ALMA_BOSU151692177480001161   
                                                   2                            TN_jstor_cspchilyoutenvi.15.2.0378   
PSYCHOSOCIAL CORRELATES OF PHYSICAL DATING VIOL... 1                                   ALMA_BOSU151692172810001161   
                                                   2                              TN_sagej10.1177_0886260509336958   
CIRCUMSTANCES SURROUNDING MALE SEXUAL ASSAULT A... 1                              TN_sagej10.1177_0886260508325488   
                                                   2                                   ALMA_BOSU151692164720001161   
SUCCESSFUL AGING FROM THE PERSPECTIVE OF FAMILY... 1                                   ALMA_BOSU151692174380001161   
                                                   2                                   ALMA_BOSU121641787320001161   
PREVALENCE OF DATING VIOLENCE AND VICTIMIZATION.   1                                           TN_proquest61409394   
                                                   2                                   ALMA_BOSU151692174450001161   
FATHER-DAUGHTER INCEST.                            1                                   ALMA_BOSU121679056360001161   
                                                   2                                             dedupmrg148318511   
RESILIENCE IN INFANCY: A RELATIONAL APPROACH.      1                                   ALMA_BOSU151692185310001161   
                                                   2                          TN_tayfranc10.1080/00377310309517704   
...                                                                                                            ...   
DEMOGRAPHIC PROFILE, TRENDS, AND COMMENTARY.       1                                   ALMA_BOSU151687773660001161   
                                                   2                                         T

In [17]:
cr_df.shape

(4916, 7)

In [ ]:
d = {}
c = 0
for k,group in by_search:
    print()
    print(k)
    print(group.columns)
    group = group[['pFulltext', 'pToplevel','pType','sType']]
    c+= 1
    print(group.groupby('pType').count())
    

In [ ]:
comp_results

In [ ]:
rt_ = comp_results[['Search','pType','sType']]
rt_.head(75)

In [16]:
pd(help)

Traceback (most recent call last):
  File "/Users/jammerman/anaconda/envs/py3x/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 3066, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-17-2a125e2f02db>", line 1, in <module>
    pd(help)
TypeError: 'module' object is not callable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jammerman/anaconda/envs/py3x/lib/python3.5/site-packages/IPython/core/interactiveshell.py", line 1877, in showtraceback
    stb = value._render_traceback_()
AttributeError: 'TypeError' object has no attribute '_render_traceback_'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/jammerman/anaconda/envs/py3x/lib/python3.5/site-packages/IPython/core/ultratb.py", line 970, in get_records
    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



TypeError: 'NoneType' object is not iterable